In [1]:
import pandas as pd
import numpy as np
import json
from collections import Counter
from catboost import CatBoostClassifier
from xgboost.sklearn import XGBClassifier
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.metrics import make_scorer, f1_score, accuracy_score, recall_score, precision_score, classification_report, precision_recall_fscore_support
import itertools
from string import punctuation
from gensim.parsing.preprocessing import STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.sparse import hstack
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from collections import Counter
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn import neighbors
from sklearn import ensemble
from sklearn import neural_network
from sklearn import linear_model
import os


/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# file used to write preserve the results of the classfier
# confusion matrix and precision recall fscore matrix

def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.tight_layout()
    return plt

In [3]:
##saving the classification report
def pandas_classification_report(y_true, y_pred):
    metrics_summary = precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred)
    cm = confusion_matrix(y_true, y_pred)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    
    avg = list(precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred,
            average='macro'))
    avg.append(accuracy_score(y_true, y_pred, normalize=True))
    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support','accuracy']
    list_all=list(metrics_summary)
    list_all.append(cm.diagonal())
    class_report_df = pd.DataFrame(
        list_all,
        index=metrics_sum_index)

    support = class_report_df.loc['support']
    total = support.sum() 
    avg[-2] = total

    class_report_df['avg / total'] = avg

    return class_report_df.T

In [4]:
from commen_preprocess import *

....start....cleaning


In [5]:
eng_train_dataset = pd.read_csv('../AMI@EVALITA2018/en_training.tsv', sep='\t')
eng_test_dataset = pd.read_csv('../AMI@EVALITA2018/en_testing.tsv', sep='\t')
#eng_train_dataset = pd.read_csv('../AMI@EVALITA2018/my_pd_train.tsv', sep='\t')
#eng_test_dataset = pd.read_csv('../AMI@EVALITA2018/my_pd_test.tsv', sep='\t')

eng_train_dataset = eng_train_dataset.sample(frac=1).reset_index(drop=True)


In [6]:
eng_train_dataset

,id,text,misogynous,misogyny_category,target
0,2813,"I don't need anyone on my dick, I stay on my o...",0,0,0
1,2193,NBC Rachel Maddcow not concerned about Rape &;...,0,0,0
2,899,Stop sayin you have resting bitch face... Bitc...,1,discredit,active
3,1941,@AnYaMemories @juthee_alam @TheSomyaSeth How d...,0,0,0
4,2666,People- how long before your heart and liver g...,0,0,0
5,1246,slut - derogatory term for a female who will f...,1,discredit,passive
6,2707,Deleted again. Working to get it back again ht...,0,0,0
7,1849,Sometimes you gotta beat ya dick like it stole...,0,0,0
8,2520,When you're chillin and the Taco Bell hits you...,0,0,0
9,2963,RT @taxo__: When it's Friday &; you're feeling...,0,0,0


In [7]:
eng_train_dataset['misogynous'].value_counts()

0    2215
1    1785
Name: misogynous, dtype: int64

In [8]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
import os
#### give the path to the glove model 
#### NOTE: this is
GLOVE_MODEL_FILE = "../../LEAM-master/glove.twitter.27B/glove.twitter.27B.200d.txt"
print(os.path.isfile(GLOVE_MODEL_FILE))

import numpy as np

##function for loading Glove Model
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r', encoding='utf8')
    model = {}
    i=0
    for line in f:
        i=i+1
        splitLine = line.split(' ')
        word = splitLine[0]
        embedding = np.asarray(splitLine[1:], dtype='float32')
        model[word] = embedding
        if(i%10000==0):
            print("count"+str(i))
    print("Done.",len(model)," words loaded!")
    return model



EMBEDDING_DIM = 200
word2vec_model = loadGloveModel(GLOVE_MODEL_FILE)

True
Loading Glove Model
count10000
count20000
count30000
count40000
count50000
count60000
count70000
count80000
count90000
count100000
count110000
count120000
count130000
count140000
count150000
count160000
count170000
count180000
count190000
count200000
count210000
count220000
count230000
count240000
count250000
count260000
count270000
count280000
count290000
count300000
count310000
count320000
count330000
count340000
count350000
count360000
count370000
count380000
count390000
count400000
count410000
count420000
count430000
count440000
count450000
count460000
count470000
count480000
count490000
count500000
count510000
count520000
count530000
count540000
count550000
count560000
count570000
count580000
count590000
count600000
count610000
count620000
count630000
count640000
count650000
count660000
count670000
count680000
count690000
count700000
count710000
count720000
count730000
count740000
count750000
count760000
count770000
count780000
count790000
count800000
count810000
count820000


In [16]:
### stopwords and punctuations are not removed but text is cleaned and stemmed more details in the commen_preprocess.py file
def glove_tokenize_norem(text):
    #text = tokenizer(text)
    text=clean(text, remove_stopwords=False, remove_punctuations=False)
    words = text.split()
    words =[ps.stem(word) for word in words]
    return words

####stopwords and punctuations are removed along with that text is cleaned ans stemmed
def glove_tokenize(text):
    #text = tokenizer(text)
    text=clean(text, remove_stopwords=False, remove_punctuations=False)
    text = ''.join([c for c in text if c not in punctuation])
    words = text.split()
    words = [word for word in words if word not in STOPWORDS]
    words =[ps.stem(word) for word in words]
    return words

def glove_tokenize_embed(text):
    #text = tokenizer(text)
    text=clean(text, remove_stopwords=False, remove_punctuations=False)
    text = ''.join([c for c in text if c not in punctuation])
    words = text.split()
    words = [word for word in words if word not in STOPWORDS]
    return words

def glove_tokenize_vocab(text):
    #text = tokenizer(text)
    text=clean(text, remove_stopwords=False, remove_punctuations=False)
    words = text.split()
    return words


In [9]:

def convert_class_label(input_text):
    if input_text==1:
        return 'misogyny'
    elif input_text==0:
        return 'non-misogyny'
    else:
        print('Wrong Input', input_text)
        sys.exit(0)

In [11]:
# pd_train = pd.DataFrame(columns=['id','misogynous','text'])
eng_train_dataset["text"].replace('', np.nan, inplace=True)
eng_train_dataset.dropna(subset=['text'], inplace=True)

pd_train_binary = eng_train_dataset[['id','misogynous','text','misogyny_category','target']]
pd_train_category = eng_train_dataset[['id','misogynous','text','misogyny_category']]
pd_train_target = eng_train_dataset[['id','misogynous','text','target']]
pd_test = eng_test_dataset[['id','text']]

pd_train_category = pd_train_category.loc[pd_train_category['misogynous'] == 1]
pd_train_target = pd_train_target.loc[pd_train_target['misogynous'] == 1]
pd_train_target.drop(['misogynous'], axis=1)                                      
pd_train_category.drop(['misogynous'], axis=1)                                      

# pd_train['class'] =pd_train.apply(lambda row: convert_class_label(row['misogynous']), axis=1)

pd_train_binary['class'] = pd_train_binary['misogynous']
pd_train_category['class'] = pd_train_category['misogyny_category']
pd_train_target['class'] = pd_train_target['target']

# for count, each in enumerate(train_data):
#     try:
#         pd_train.loc[count]  = [each['id'], convert_class_label(each['CounterSpeech']), each['Community'],each['Category'],each['commentText']]
#     except:
#         pass
print('Training Data Loading Completed...')

Training Data Loading Completed...


/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [12]:
def get_data(pd_train):
    comments=pd_train['text'].values
    labels=pd_train['class'].values
    
    list_comment=[]
    for comment,label in zip(comments,labels):
        temp={}
        temp['text']=comment
        temp['label']=label
        list_comment.append(temp)
    return list_comment 

def get_data_test(pd_test):
    comments=pd_test['text'].values
    list_comment=[]
    for comment in comments:
        temp={}
        temp['text']=comment
        list_comment.append(temp)
    return list_comment 


In [13]:
vocab={}
def create_vocab(data):
    comments = get_data(data)
    for comment in comments:
        words=glove_tokenize_vocab(comment['text'])
        for word in words:
            if word in vocab.keys():
                vocab[word]=vocab[word]+1 
            else:
                vocab[word]=1
                
        

### Google Universal Sentence Encoder configuration
###### prerequisite: tensorflow version >=1.7

In [14]:
import tensorflow as tf
import tensorflow_hub as hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
embed = hub.Module(module_url)
config = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=12,
                       allow_soft_placement=True, device_count = {'CPU': 12})

def get_embeddings(messages):
      
    with tf.Session(config=config) as session:
            session.run([tf.global_variables_initializer(), tf.tables_initializer()])
            message_emb = session.run(embed(messages))
            
    print("ending")
    return np.array(message_emb)

/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/2'.
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/2'.
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_0:0 from checkpoint b'/tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_0
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_1:0 from checkpoint b'/tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_1
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_10:0 from checkpoint b'/tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_10
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_11:0 from checkpoint b'/tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/var

In [24]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

TOKENIZER = glove_tokenize
#google encoding used where text is cleaned  
def gen_data_google(data):
    comments = get_data(data)
    X, y = [], []
    for comment in comments:
        y.append(comment['label'])
        #X.append(tokenizer(comment['text']))
        X.append(clean(comment['text'], remove_stopwords=True, remove_punctuations=True))
    #TFIDF_feature = 'bpe_text'

    #Word Level Features
    X =get_embeddings(X)
    # print y
    #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    
    return X, y

#google encoding used where text is not cleaned 
def gen_data_google2(data):
    comments = get_data(data)
    X, y = [],[]
    for comment in comments:
        y.append(comment['label'])
        X.append(clean(comment['text'], remove_stopwords=False, remove_punctuations=False))
    #Word Level Features
    X =get_embeddings(X)
    #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    return X,y

###get data male,female
def male_female(data):
    comments = get_data(data)
    X = []
    for comment in comments:
        X.append(np.array([comment['male'],comment['female']]))
    #Word Level Features
    
    #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    return np.array(X)



### tfidf feature generation was used here where stopwords and punctuations are removed 
def gen_data_new_tfidf(data):
    comments = get_data(data)
    comments_test=get_data_test(pd_test)
    X, y = [], []
    for comment in comments:
        y.append(comment['label'])
        X.append(comment['text'])

    X1=[]
    for comment in comments_test:
        X1.append(comment['text'])


    #Word Level Features
    word_vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(1,3),
                min_df=1, 
                strip_accents='unicode',
                #smooth_idf=1,
                analyzer='word', 
                stop_words='english',
                tokenizer=TOKENIZER,             
                max_features=5000)
    
    
    #charlevel features new
    char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    #stop_words='english',
    ngram_range=(2, 6),
    max_features=10000)
    word_vectorizer.fit(X+X1)
    char_vectorizer.fit(X+X1)
    
    with open('tfidf_word_vectorizer.pk', 'wb') as fout:
         pickle.dump(word_vectorizer,fout)

    with open('tfidf_char_vectorizer.pk', 'wb') as fout:
        pickle.dump(char_vectorizer,fout)
    
    test_word_features = word_vectorizer.transform(X)
    test_char_features = char_vectorizer.transform(X)
    X = list(hstack([test_char_features, test_word_features]).toarray())
    #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    return X, y

### tfidf feature generation was used here where stopwords and punctuations are not removed 
def gen_data_new_tfidf2(data):
    comments = get_data(data)
    X, y = [], []
    for comment in comments:
        y.append(comment['label'])
        X.append(comment['text'])


    #Word Level Features
    word_vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(1,3),
                min_df=1, 
                strip_accents='unicode',
                #smooth_idf=1,
                analyzer='word', 
                #stop_words='english',
                tokenizer=glove_tokenize_norem,             
                max_features=5000)
    
    
    #charlevel features new
    char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    #stop_words='english',
    ngram_range=(2, 6),
    max_features=10000)
    
    word_vectorizer.fit(X)
    char_vectorizer.fit(X)
    
    with open('tfidf_word_vectorize_noclean.pk', 'wb') as fout:
         pickle.dump(word_vectorizer,fout)

    with open('tfidf_char_vectorizer_noclean.pk', 'wb') as fout:
         pickle.dump(char_vectorizer,fout)
        
    test_word_features = word_vectorizer.transform(X)
    test_char_features = char_vectorizer.transform(X)
    X = list(hstack([test_char_features, test_word_features]).toarray())
    #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    
    return X, y

## combination of not cleaned google encodings and tfidf features where stopwords and punctuations are not removed 
def combine_tf_google_rem(data):
    X,_=gen_data_google(data)
    X1,y=gen_data_new_tfidf(data)
#     X1,y=gen_data_old_tfidf()
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y

## combination of cleaned google encodings and tfidf features where stopwords and punctuations are ssremoved 
def combine_tf_google_norem(data):
    X,_=gen_data_google2(data)
    X1,y=gen_data_new_tfidf2(data)
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y

def combine_tf_rem_google_norem(data):
    X,_=gen_data_google2(data)
    X1,y=gen_data_new_tfidf(data)
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y

def combine_tf_norem_google_rem(data):
    X,_=gen_data_google(data)
    X1,y=gen_data_new_tfidf2(data)
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y

def gen_data_embed(data):
    comments = get_data(data)
    X, y = [], []
    for comment in comments:
        words = glove_tokenize_embed(comment['text'].lower())
        emb = np.zeros(EMBEDDING_DIM)
        for word in words:
            try:
                emb += word2vec_model[word]
            except:
                pass
        if len(words)!=0:
            emb /= len(words)
        X.append(emb)
        y.append(comment['label'])

    return X, y


def combine_tf_rem_google_norem_embed(data):
    X,_=gen_data_google2(data)
    X1,y=gen_data_new_tfidf(data)
    X2,_=gen_data_embed(data)
    X=np.concatenate((np.array(X), np.array(X1),np.array(X2)), axis=1)
    return X,y



###old tfidf

def gen_data_old_tfidf(data):
    comments = get_data(data)
    X, y = [], []
    for comment in comments:
        y.append(comment['label'])
        X.append(comment['text'])
    with open('../tfidf_word_vectorizer.pk', 'rb') as fin:
        word_vectorizer = pickle.load(fin)

    with open('../tfidf_char_vectorizer.pk', 'rb') as fin:
        char_vectorizer = pickle.load(fin)


    
    word_vectorizer.fit(X)
    char_vectorizer.fit(X)
    
    test_word_features = word_vectorizer.transform(X)
    test_char_features = char_vectorizer.transform(X)
    X = list(hstack([test_char_features, test_word_features]).toarray())
    #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    
    return X, y

def select_comments_whose_embedding_exists(flag):
    # selects the comments as in mean_glove_embedding method
    # Processing
    comments = get_data(flag)
    X, Y = [], []
    comment_return = []
    for tweet in comments:
        #print(tweet)
        _emb = 0
        words = TOKENIZER(tweet['text'].lower())
        for w in words:
            #print(w)
            if w in word2vec_model and w is not None:  # Check if embeeding there in GLove model
                _emb+=1
        if _emb:   # Not a blank tweet
            comment_return.append(tweet)
    print('Comments selected:', len(comment_return))
    return comment_return

def gen_data():
    comments = select_comments_whose_embedding_exists(0)
    X, y = [], []
    for comment in comments:
        words = glove_tokenize(comment['text'].lower())
        emb = numpy.zeros(EMBEDDING_DIM)
        for word in words:
            try:
                emb += word2vec_model[word]
            except:
                pass
        emb /= len(words)
        X.append(emb)
        y.append(comment['label'])

    # print y
    y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    print
    return X, y

## combination of not cleaned google encodings and tfidf features where stopwords and punctuations are not removed 
def combine_tf_google_glove_rem():
    X,_=gen_data_google()
    X1,y=gen_data_new_tfidf()
#     X1,y=gen_data_old_tfidf()
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y

In [18]:
SCALE_POS_WEIGHT=None

In [19]:
from imblearn.over_sampling import ADASYN
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
def get_model(m_type=None):
    if not m_type:
        print("ERROR: Please specify a model type!")
        return None
    if m_type == 'decision_tree_classifier':
        logreg = tree.DecisionTreeClassifier(class_weight='balanced')
    elif m_type == 'gaussian':
        logreg = GaussianNB()
    elif m_type == 'logistic_regression':
        logreg = LogisticRegression(class_weight='balanced',n_jobs=10, random_state=42)
    elif m_type == 'MLPClassifier':
#         logreg = neural_network.MLPClassifier((500))
        logreg = neural_network.MLPClassifier(random_state=42,early_stopping=True)
    elif m_type == 'KNeighborsClassifier':
#         logreg = neighbors.KNeighborsClassifier(n_neighbors = 10)
        logreg = neighbors.KNeighborsClassifier()
    elif m_type == 'ExtraTreeClassifier':
        logreg = tree.ExtraTreeClassifier()
    elif m_type == 'ExtraTreeClassifier_2':
        logreg = ensemble.ExtraTreesClassifier()
    elif m_type == 'RandomForestClassifier':
        logreg = ensemble.RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=12, max_depth=3)
    elif m_type == 'SVC':
        logreg = LinearSVC(class_weight='balanced');
    elif m_type == 'Catboost':
        logreg = CatBoostClassifier(use_best_model=False, random_state=42, scale_pos_weight=SCALE_POS_WEIGHT)
#         logreg = CatBoostClassifier(scale_pos_weight=0.8, random_seed=42,);
    elif m_type == 'XGB_classifier':
#         logreg=XGBClassifier(silent=False,eta=0.1,objective='binary:logistic',max_depth=5,min_child_weight=0,gamma=0.2,subsample=0.8, colsample_bytree = 0.8,scale_pos_weight=1,n_estimators=500,reg_lambda=3,nthread=12)
        logreg=XGBClassifier(silent=False,objective='binary:logistic',scale_pos_weight=0.8,reg_lambda=3,nthread=12, random_state=42)
    elif m_type == 'binny_test':
        clf1 = ensemble.RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=12, max_depth=6,max_features='auto')
        clf2 = tree.DecisionTreeClassifier(random_state=42, class_weight='balanced',max_depth=6)
        clf3 = LogisticRegression(class_weight='balanced',penalty="l2",C=0.1, dual=True, random_state=42, n_jobs=3)
        clf4 = XGBClassifier(silent=False,objective='binary:logistic',scale_pos_weight=0.8,reg_lambda=3,nthread=12, random_state=42)
        est_list = [('lr', clf1), ('rf', clf2), ('gnb', clf3), ('xgb', clf4)]
        logreg = ensemble.VotingClassifier(est_list,voting='soft',n_jobs=6)
    else:
        print("give correct model")
    print(logreg)
    return logreg

def get_feature(f_type=None,data=None):
    if not f_type:
        print("ERROR: Please specify a model type!")
        return None
    if f_type == 'google_not_preprocess':
        X,y=gen_data_google2(data)
    elif f_type == 'word_to_vec_embed':
        X,y=gen_data_embed(data)
    elif f_type == 'google_preprocess':
        X,y=gen_data_google(data)
    elif f_type == 'tfidf_not_preprocess':
        X,y=gen_data_new_tfidf2(data)
    elif f_type == 'tfidf_preprocess':
        X,y=gen_data_new_tfidf(data)
    elif f_type == 'google_preprocess_tfidf_preprocess':
        X,y=combine_tf_google_rem(data)
    elif f_type == 'google_nopreprocess_tfidf_nopreprocess':
        X,y=combine_tf_google_norem(data)
    elif f_type == 'google_preprocess_tfidf_nopreprocess':
        X,y=combine_tf_norem_google_rem(data)
    elif f_type == 'google_nopreprocess_tfidf_preprocess':
        X,y=combine_tf_rem_google_norem(data)
    elif f_type == 'google_nopreprocess_tfidf_preprocess_embed':
        X,y=combine_tf_rem_google_norem_embed(data)
    else:
        print("give correct feature selection")    
    print(f_type)
    return X,y



In [28]:
# from imblearn.combine import SMOTETomek 
from sklearn.metrics import accuracy_score
import joblib
from sklearn.model_selection import StratifiedKFold as skf

def binny_classifier_run(X,y,model,model_name,label_map,img_name,report_name,save_model=False):
    Classifier_Train_X = np.array(X, copy=False)
    Classifier_Train_Y = y
    temp=[]
    for data in Classifier_Train_Y:
        temp.append(label_map[data])
    SCALE_POS_WEIGHT=temp.count(0)/temp.count(1)
    
    Classifier_Train_Y=np.array(temp)
    
    
    model_featureSelection = SelectFromModel(ensemble.RandomForestClassifier(n_estimators=50, class_weight='balanced', 
                                                                                      n_jobs=12, max_depth=3))
    print('Before Num features=',Classifier_Train_X.shape[1], Counter(Classifier_Train_Y))
    Classifier_Train_X = model_featureSelection.fit_transform(Classifier_Train_X,Classifier_Train_Y)
    print('After Num features=',Classifier_Train_X.shape[1])

    
    if(save_model==True):
        Classifier=model
        Classifier.fit(Classifier_Train_X, Classifier_Train_Y)
        filename = 'task1/'+model_name+'_task_1.joblib.pkl'
        joblib.dump(Classifier, filename, compress=9)
        filename1 = 'task1/'+model_name+'_select_features_task1.joblib.pkl'
        joblib.dump(model_featureSelection, filename1, compress=9)
        t1 = pd.read_csv('../AMI@EVALITA2018/en_testing_tagged.tsv',sep='\t')
        t1['label']=t1['TAG']
        t1['class']=t1['TAG']
        t1_X,t1_y=get_feature(f_type=feature_model,data=t1)
        print('Before Num features=',t1_X.shape[1], Counter(t1_y))
        t1_X = model_featureSelection.transform(t1_X)
        print('After Num features=',t1_X.shape[1])
        Final_test_X = np.array(t1_X, copy=False)
        Final_test_Y = np.array(t1_y)
        Final_pred_y = Classifier.predict(Final_test_X)
        print("BINNY DATA:")
        print(classification_report( Final_test_Y, Final_pred_y ))
        print('accuracy_test_BINNY:',accuracy_score(Final_test_Y, Final_pred_y))

    else:
        kf = skf(n_splits=10,shuffle=True)
        y_total_preds=[] 
        y_total=[]
        count=0

        for train_index, test_index in kf.split(Classifier_Train_X,Classifier_Train_Y):
            X_train, X_test = Classifier_Train_X[train_index], Classifier_Train_X[test_index]
            y_train, y_test = Classifier_Train_Y[train_index], Classifier_Train_Y[test_index]
            classifier=model 
            classifier.fit(X_train,y_train)
            y_preds = classifier.predict(X_test)
            for ele in y_test:
                y_total.append(ele)
            for ele in y_preds:
                y_total_preds.append(ele)
            y_pred_train = classifier.predict(X_train)
            count=count+1       
            print('accuracy_train:',accuracy_score(y_train, y_pred_train),'accuracy_test:',accuracy_score(y_test, y_preds))
            print('TRAINING:')
            print(classification_report( y_train, y_pred_train ))
            print("TESTING:")
            print(classification_report( y_test, y_preds ))
            
        report = classification_report( y_total, y_total_preds )
        cm=confusion_matrix(y_total, y_total_preds)
        plt=plot_confusion_matrix(cm,normalize= True,target_names = ['non_misgynous','misgynous'],title = "Confusion Matrix")
        plt.savefig('task1'+model_name+'_'+img_name)
        print(model)
        print(report)
        print(accuracy_score(y_total, y_total_preds))
        df_result=pandas_classification_report(y_total,y_total_preds)
        df_result.to_csv('task1'+model_name+'_'+report_name,  sep=',')


In [29]:
feature_model = 'google_nopreprocess_tfidf_preprocess_embed'
img_name = 'cm.png'
report_name = 'report.csv'

In [30]:
data_name= 'pd_train_binary'

if(data_name=='pd_train_binary'):
    X,y=get_feature(f_type=feature_model,data=pd_train_binary)
    label_map = {
             1: 1,
             0: 0
                 }
elif(data_name=='pd_train_category'):
    X,y=get_feature(f_type=feature_model,data=pd_train_category)
    label_map = {
            'discredit': 0,
            'sexual_harassment': 1,
            'stereotype': 2,
            'dominance': 3,
            'derailing': 4
        }
elif(data_name=='pd_train_target'):
    X,y=get_feature(f_type=feature_model,data=pd_train_target)
    label_map = {
             'active': 1,
             'passive': 0
         }

else:
    print('give correct data')
    

ending
google_nopreprocess_tfidf_preprocess_embed


In [31]:
#list_of_model = ['decision_tree_classifier', 'gaussian', 'logistic_regression', 'MLPClassifier', 'RandomForestClassifier',
#                 'SVC', 'Catboost', 'XGB_classifier']
list_of_model = ['Catboost','XGB_classifier','logistic_regression']

for each_model in list_of_model:
    model=get_model(m_type=each_model)
    binny_classifier_run(X,y,model,each_model,label_map,img_name,report_name,save_model=True)


Before Num features= 15712 Counter({0: 2215, 1: 1785})
After Num features= 253
Learning rate set to 0.024803
0:	learn: 0.6807219	total: 40.1ms	remaining: 40s
1:	learn: 0.6708206	total: 74.5ms	remaining: 37.2s
2:	learn: 0.6602255	total: 105ms	remaining: 34.9s
3:	learn: 0.6502977	total: 133ms	remaining: 33.1s
4:	learn: 0.6410137	total: 161ms	remaining: 32s
5:	learn: 0.6308812	total: 189ms	remaining: 31.3s
6:	learn: 0.6233346	total: 215ms	remaining: 30.5s
7:	learn: 0.6159610	total: 246ms	remaining: 30.5s
8:	learn: 0.6082152	total: 275ms	remaining: 30.3s
9:	learn: 0.6001265	total: 301ms	remaining: 29.8s
10:	learn: 0.5923223	total: 329ms	remaining: 29.6s
11:	learn: 0.5850889	total: 357ms	remaining: 29.4s
12:	learn: 0.5783535	total: 386ms	remaining: 29.3s
13:	learn: 0.5729422	total: 411ms	remaining: 29s
14:	learn: 0.5668037	total: 441ms	remaining: 28.9s
15:	learn: 0.5612591	total: 468ms	remaining: 28.8s
16:	learn: 0.5565169	total: 495ms	remaining: 28.6s
17:	learn: 0.5515766	total: 525ms	rema

159:	learn: 0.3767837	total: 4.33s	remaining: 22.8s
160:	learn: 0.3763199	total: 4.36s	remaining: 22.7s
161:	learn: 0.3759642	total: 4.39s	remaining: 22.7s
162:	learn: 0.3755695	total: 4.41s	remaining: 22.7s
163:	learn: 0.3750871	total: 4.44s	remaining: 22.6s
164:	learn: 0.3746753	total: 4.47s	remaining: 22.6s
165:	learn: 0.3742987	total: 4.5s	remaining: 22.6s
166:	learn: 0.3738685	total: 4.52s	remaining: 22.6s
167:	learn: 0.3732836	total: 4.55s	remaining: 22.6s
168:	learn: 0.3727640	total: 4.58s	remaining: 22.5s
169:	learn: 0.3724209	total: 4.61s	remaining: 22.5s
170:	learn: 0.3717749	total: 4.63s	remaining: 22.5s
171:	learn: 0.3712772	total: 4.66s	remaining: 22.4s
172:	learn: 0.3709776	total: 4.68s	remaining: 22.4s
173:	learn: 0.3706704	total: 4.71s	remaining: 22.4s
174:	learn: 0.3702439	total: 4.74s	remaining: 22.3s
175:	learn: 0.3699020	total: 4.76s	remaining: 22.3s
176:	learn: 0.3695510	total: 4.79s	remaining: 22.3s
177:	learn: 0.3690202	total: 4.82s	remaining: 22.2s
178:	learn: 0

320:	learn: 0.3205564	total: 8.53s	remaining: 18s
321:	learn: 0.3202377	total: 8.56s	remaining: 18s
322:	learn: 0.3199703	total: 8.58s	remaining: 18s
323:	learn: 0.3197922	total: 8.61s	remaining: 18s
324:	learn: 0.3194197	total: 8.63s	remaining: 17.9s
325:	learn: 0.3191960	total: 8.65s	remaining: 17.9s
326:	learn: 0.3186100	total: 8.68s	remaining: 17.9s
327:	learn: 0.3183780	total: 8.71s	remaining: 17.8s
328:	learn: 0.3180751	total: 8.74s	remaining: 17.8s
329:	learn: 0.3177007	total: 8.76s	remaining: 17.8s
330:	learn: 0.3173519	total: 8.79s	remaining: 17.8s
331:	learn: 0.3169965	total: 8.81s	remaining: 17.7s
332:	learn: 0.3167695	total: 8.84s	remaining: 17.7s
333:	learn: 0.3165371	total: 8.86s	remaining: 17.7s
334:	learn: 0.3163419	total: 8.89s	remaining: 17.6s
335:	learn: 0.3160390	total: 8.91s	remaining: 17.6s
336:	learn: 0.3156659	total: 8.94s	remaining: 17.6s
337:	learn: 0.3151807	total: 8.97s	remaining: 17.6s
338:	learn: 0.3149029	total: 9s	remaining: 17.5s
339:	learn: 0.3146657	t

480:	learn: 0.2680291	total: 12.7s	remaining: 13.7s
481:	learn: 0.2678184	total: 12.7s	remaining: 13.7s
482:	learn: 0.2675411	total: 12.8s	remaining: 13.7s
483:	learn: 0.2671193	total: 12.8s	remaining: 13.6s
484:	learn: 0.2668452	total: 12.8s	remaining: 13.6s
485:	learn: 0.2666058	total: 12.8s	remaining: 13.6s
486:	learn: 0.2661925	total: 12.9s	remaining: 13.6s
487:	learn: 0.2657726	total: 12.9s	remaining: 13.5s
488:	learn: 0.2655118	total: 12.9s	remaining: 13.5s
489:	learn: 0.2653282	total: 12.9s	remaining: 13.5s
490:	learn: 0.2649655	total: 13s	remaining: 13.5s
491:	learn: 0.2646690	total: 13s	remaining: 13.4s
492:	learn: 0.2643596	total: 13s	remaining: 13.4s
493:	learn: 0.2640415	total: 13.1s	remaining: 13.4s
494:	learn: 0.2637755	total: 13.1s	remaining: 13.3s
495:	learn: 0.2634494	total: 13.1s	remaining: 13.3s
496:	learn: 0.2630479	total: 13.1s	remaining: 13.3s
497:	learn: 0.2627917	total: 13.2s	remaining: 13.3s
498:	learn: 0.2624624	total: 13.2s	remaining: 13.2s
499:	learn: 0.2622

641:	learn: 0.2237479	total: 16.9s	remaining: 9.44s
642:	learn: 0.2233537	total: 16.9s	remaining: 9.41s
643:	learn: 0.2229622	total: 17s	remaining: 9.39s
644:	learn: 0.2226125	total: 17s	remaining: 9.36s
645:	learn: 0.2222921	total: 17s	remaining: 9.34s
646:	learn: 0.2221682	total: 17.1s	remaining: 9.31s
647:	learn: 0.2218614	total: 17.1s	remaining: 9.28s
648:	learn: 0.2215247	total: 17.1s	remaining: 9.26s
649:	learn: 0.2213935	total: 17.1s	remaining: 9.23s
650:	learn: 0.2212096	total: 17.2s	remaining: 9.21s
651:	learn: 0.2210771	total: 17.2s	remaining: 9.18s
652:	learn: 0.2208112	total: 17.2s	remaining: 9.15s
653:	learn: 0.2204886	total: 17.2s	remaining: 9.13s
654:	learn: 0.2202713	total: 17.3s	remaining: 9.1s
655:	learn: 0.2200946	total: 17.3s	remaining: 9.07s
656:	learn: 0.2197678	total: 17.3s	remaining: 9.04s
657:	learn: 0.2195565	total: 17.4s	remaining: 9.02s
658:	learn: 0.2194268	total: 17.4s	remaining: 8.99s
659:	learn: 0.2191647	total: 17.4s	remaining: 8.96s
660:	learn: 0.21896

...

961:	learn: 0.1658717	total: 25.2s	remaining: 997ms
962:	learn: 0.1655993	total: 25.3s	remaining: 971ms
963:	learn: 0.1655081	total: 25.3s	remaining: 944ms
964:	learn: 0.1653591	total: 25.3s	remaining: 918ms
965:	learn: 0.1651841	total: 25.3s	remaining: 892ms
966:	learn: 0.1650771	total: 25.4s	remaining: 865ms
967:	learn: 0.1649805	total: 25.4s	remaining: 839ms
968:	learn: 0.1648257	total: 25.4s	remaining: 813ms
969:	learn: 0.1646451	total: 25.4s	remaining: 787ms
970:	learn: 0.1643603	total: 25.5s	remaining: 761ms
971:	learn: 0.1642201	total: 25.5s	remaining: 734ms
972:	learn: 0.1640730	total: 25.5s	remaining: 708ms
973:	learn: 0.1638722	total: 25.5s	remaining: 682ms
974:	learn: 0.1636677	total: 25.6s	remaining: 656ms
975:	learn: 0.1635385	total: 25.6s	remaining: 630ms
976:	learn: 0.1633817	total: 25.6s	remaining: 603ms
977:	learn: 0.1631303	total: 25.7s	remaining: 577ms
978:	learn: 0.1630030	total: 25.7s	remaining: 551ms
979:	learn: 0.1628099	total: 25.7s	remaining: 525ms
980:	learn: 

/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


After Num features= 257


/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.
  " = {}.".format(self.n_jobs))


ending
google_nopreprocess_tfidf_preprocess_embed
Before Num features= 15712 Counter({1: 27, 0: 23})
After Num features= 257
BINNY DATA:
             precision    recall  f1-score   support

          0       0.62      0.70      0.65        23
          1       0.71      0.63      0.67        27

avg / total       0.67      0.66      0.66        50

accuracy_test_BINNY: 0.66
